In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/ML_vui_test.csv


In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc, accuracy_score


In [3]:
#data = pd.read_excel('/kaggle/input/ML_vui_test.csv')
#pd.options.display.float_format = '{:,.4f}'.format
# Some data in ML_vui_test file has numeric comma format("abc,xyz"), which somehow makes sklearn failed to transform. 
# Reformat them into dot format (abc.xyz)
data = pd.read_csv("/kaggle/input/ML_vui_test.csv", sep=";")
data.head()
#data.loc[pd.notnull(data.P_BIRTHDAY)].head()

,GENDER,P_BIRTHDAY,Order,Items,GMV,AIV,AOV,Last Purchase,Num_of_cate,Check_mua_TP
0,NaN,NaN,1,1,20000.0,20000.0,20000.0,344.0,1,0
1,NaN,NaN,1,1,20000.0,20000.0,20000.0,344.0,1,0
2,NaN,NaN,1,1,20000.0,20000.0,20000.0,302.0,1,0
3,NaN,NaN,1,1,20000.0,20000.0,20000.0,350.0,1,0
4,NaN,NaN,1,1,20000.0,20000.0,20000.0,303.0,1,0


In [4]:
data['GENDER'].fillna('UNDEFINE',inplace=True)
#data['P_BIRTHDAY'].fillna('2010-01-01 00:00:00',inplace=True)
data['P_BIRTHDAY'].fillna('2010-01-01 00:00:00',inplace=True)

In [5]:
data['P_BIRTHDAY'] = pd.to_datetime(data['P_BIRTHDAY'])
data['Age'] = pd.datetime.today().year - data['P_BIRTHDAY'].dt.year

In [6]:
data = data.drop(['P_BIRTHDAY'], axis=1)
data.head()

,GENDER,Order,Items,GMV,AIV,AOV,Last Purchase,Num_of_cate,Check_mua_TP,Age
0,UNDEFINE,1,1,20000.0,20000.0,20000.0,344.0,1,0,9
1,UNDEFINE,1,1,20000.0,20000.0,20000.0,344.0,1,0,9
2,UNDEFINE,1,1,20000.0,20000.0,20000.0,302.0,1,0,9
3,UNDEFINE,1,1,20000.0,20000.0,20000.0,350.0,1,0,9
4,UNDEFINE,1,1,20000.0,20000.0,20000.0,303.0,1,0,9


In [7]:
data.describe()

,Order,Items,GMV,AIV,AOV,Last Purchase,Num_of_cate,Check_mua_TP,Age
count,91783.000000,91783.00000,9.178300e+04,9.178300e+04,9.178300e+04,91783.000000,91783.000000,91783.000000,91783.000000
mean,4.141442,12.30887,9.716691e+06,2.112441e+06,2.560145e+06,378.228525,1.435800,0.083480,19.037153
std,10.796529,91.72151,2.834469e+07,4.166307e+06,4.570578e+06,282.977908,1.081295,0.276607,14.060966
min,1.000000,1.00000,2.000000e+00,1.000000e+00,2.000000e+00,0.000000,1.000000,0.000000,-49.000000
25%,1.000000,1.00000,1.690000e+05,7.540339e+04,1.000000e+05,149.000000,1.000000,0.000000,9.000000
50%,1.000000,2.00000,1.200000e+06,3.400000e+05,5.303265e+05,345.000000,1.000000,0.000000,19.000000
75%,3.000000,5.00000,7.249000e+06,2.245000e+06,2.890000e+06,614.000000,1.000000,0.000000,30.000000
max,731.000000,12308.00000,2.534474e+09,6.823750e+07,1.864190e+08,999.000000,8.000000,1.000000,133.000000


In [8]:
X = data.drop(['Check_mua_TP'], axis = 1)
Y = data['Check_mua_TP'].copy()
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2) #, random_state=42

In [9]:
X_num = X_train.drop("GENDER", axis = 1)

In [10]:
gender = X_train[["GENDER"]]

In [11]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
lblEnc = LabelEncoder()
ohEnc = OneHotEncoder()

In [12]:
ohEnc.fit(gender)

OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='error',
              n_values=None, sparse=True)

In [13]:
genderOnehot = ohEnc.transform(gender)

In [ ]:
## Replacement of OneHotEncoder for local environment if OneHotEncoder works improbably 
# from sklearn.preprocessing import LabelBinarizer
# genderEncoder = LabelBinarizer()
# genderOnehot = genderEncoder.fit_transform(gender)

In [14]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataTypeConverter(BaseEstimator, TransformerMixin):
    def __init__(self): # no *args or **kargs
        return
    
    def fit(self, X, y=None):
        return self # nothing else to do 
    
    def transform(self, X, y=None):
        # Convert all fields into float
        return X.astype(np.float)

floatConverter = DataTypeConverter()
X_num_test = floatConverter.transform(X_num.values)

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
numPipeline = Pipeline([
    ("obj_converter", DataTypeConverter()),
    ('std_scaler', StandardScaler()),
])
X_NumTr = numPipeline.fit_transform(X_num)

In [16]:
from sklearn.compose import ColumnTransformer 
numAttribs = list(X_num)
catAttribs = ["GENDER"]
fullPipeline = ColumnTransformer([
    ("num", numPipeline, numAttribs),
    ("cat", OneHotEncoder(), catAttribs),
    # ("cat", LabelBinarizer(), catAttribs), # If OneHotEncoder() does not work for local anaconda
 ])
dataPrepared_X_train = fullPipeline.fit_transform(X_train)
dataPrepared_X_test = fullPipeline.fit_transform(X_test)


In [17]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier( n_estimators=200, max_leaf_nodes=8, oob_score=True)

model.fit(dataPrepared_X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=8,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [21]:
predict_rf = model.predict(dataPrepared_X_test)
accuracy_rf = accuracy_score(predict_rf,y_test)
print('The accuracy score is {}'.format(accuracy_rf))
confusion_matrix_rf = confusion_matrix(y_test,predict_rf)
print(confusion_matrix_rf)

The accuracy score is 0.9456882932941112
[[16649   173]
 [  824   711]]


In [22]:
from sklearn.metrics import roc_auc_score
auc_rf = roc_auc_score(y_test, predict_rf)
print(auc_rf)

0.726454015352162
